In [1]:
import multiprocessing
import pathlib
import time

import click
import numpy as np
from COVIDScenarioPipeline.SEIR import seir, setup
from COVIDScenarioPipeline.SEIR.utils import config
from COVIDScenarioPipeline.SEIR.profile import profile_options


config.set_file('config.yml')
spatial_config = config["spatial_setup"]
spatial_base_path = pathlib.Path(spatial_config["base_path"].get())
scenario = 'Scenario1'
nsim = 10
interactive = False
write_csv = False
write_parquet = True
index = 1
ncomp = 7
S, E, I1, I2, I3, R, cumI = np.arange(ncomp)



s = setup.Setup(setup_name=config["name"].get() + "_" + str(scenario),
                        spatial_setup=setup.SpatialSetup(
                            setup_name=spatial_config["setup_name"].get(),
                            geodata_file=spatial_base_path / spatial_config["geodata"].get(),
                            mobility_file=spatial_base_path / spatial_config["mobility"].get(),
                            popnodes_key=spatial_config["popnodes"].get(),
                            nodenames_key=spatial_config["nodenames"].get()
                        ),
                        nsim=nsim,
                        npi_scenario=scenario,
                        npi_config=config["interventions"]["settings"][scenario],
                        seeding_config=config["seeding"],
                        ti=config["start_date"].as_date(),
                        tf=config["end_date"].as_date(),
                        interactive=interactive,
                        write_csv=write_csv,
                        write_parquet=write_parquet,
                        dt=config["dt"].as_number(),
                        first_sim_index = index)


print(f"""
>> Scenario: {scenario}
>> Starting {s.nsim} model runs beginning from {s.first_sim_index}
>> Setup *** {s.setup_name} *** from {s.ti} to {s.tf}
>> writing to folder : {s.datadir}{s.setup_name}
    """)
onerun_SEIR = lambda sim_id2write, s: seir.onerun_SEIR(sim_id2write, s)
onerun_SEIR_loadID = lambda sim_id2write, s, sim_id2load: seir.onerun_SEIR_loadID(sim_id2write, s, sim_id2load)

Mobility files as matrices are not recommended. Please switch soon to long form csv files.

>> Scenario: Scenario1
>> Starting 10 model runs beginning from 1
>> Setup *** minimal_Scenario1 *** from 2020-01-15 to 2020-05-31
>> writing to folder : model_output/minimal_Scenario1/minimal_Scenario1
    


In [2]:
onerun_SEIR(1,s)
onerun_SEIR_loadID(2,s,1)

ValueError: place 20002 does not exist in seeding::states_file. You can set ignore_missing=TRUE to bypass this error

In [57]:
states = pd.read_csv(s.seeding_config["states_file"].as_str(), converters={'place': lambda x: str(x)})
if (states.empty):
    raise ValueError(f"There is no entry for initial time ti in the provided seeding::states_file.")

y0 = np.zeros((ncomp, s.nnodes))

for pl in s.spatset.nodenames:
    if pl in list(states['place']):
        y0[S][s.spatset.nodenames.index(pl)] =  float(states[(states['place'] == pl) & (states['comp'] == 'S')]['amount'])
        y0[E][s.spatset.nodenames.index(pl)] =  float(states[(states['place'] == pl) & (states['comp'] == 'E')]['amount'])
        y0[I1][s.spatset.nodenames.index(pl)] = float(states[(states['place'] == pl) & (states['comp'] == 'I1')]['amount'])
        y0[I2][s.spatset.nodenames.index(pl)] = float(states[(states['place'] == pl) & (states['comp'] == 'I2')]['amount'])
        y0[I3][s.spatset.nodenames.index(pl)] = float(states[(states['place'] == pl) & (states['comp'] == 'I3')]['amount'])
        y0[R][s.spatset.nodenames.index(pl)] =  float(states[(states['place'] == pl) & (states['comp'] == 'R')]['amount'])
        y0[cumI][s.spatset.nodenames.index(pl)] = y0[I1][s.spatset.nodenames.index(pl)] + y0[I2][s.spatset.nodenames.index(pl)] + y0[I3][s.spatset.nodenames.index(pl)]
    elif s.seeding_config["ignore_missing"].get():
        print(f'WARNING: State load does not exist for node {pl}, assuming fully susceptible population')
        y0[S, s.spatset.nodenames.index(pl)] = s.popnodes[s.spatset.nodenames.index(pl)]
    else:
        raise ValueError(f"place {pl} does not exist in seeding::states_file. You can set ignore_missing=TRUE to bypass this error")

In [51]:
states['comp'] == 'S'

0         True
1         True
2         True
3         True
4         True
         ...  
18847    False
18848    False
18849    False
18850    False
18851    False
Name: comp, Length: 18852, dtype: bool

In [45]:
y0[:,1]

array([2000.,    0.,    0.,    0.,    0.,    0.,    0.])

In [12]:
states['time'][0]

Timestamp('2020-01-15 00:00:00')

In [2]:
y0, i = setup.seeding_draw(s,1)

In [3]:
y0

array([[156520.        ,   2000.        ],
       [  1275.47327772,      0.        ],
       [   365.35379596,      0.        ],
       [   353.5132001 ,      0.        ],
       [   306.51315982,      0.        ],
       [  3489.1465664 ,      0.        ],
       [  4514.52672228,      0.        ]])

In [6]:
y0, i = setup.seeding_load(s,1)

In [7]:
y0

array([[156520.        ,   2000.        ],
       [  1275.47327772,      0.        ],
       [   365.35379596,      0.        ],
       [   353.5132001 ,      0.        ],
       [   306.51315982,      0.        ],
       [  3489.1465664 ,      0.        ],
       [  4514.52672228,      0.        ]])